# Ensemble Classifiers

## Import required libraries

In [37]:
import numpy as np
import pandas as pd
import time
import pickle # allows for model to be saved/load to file
import xgboost
from sklearn.ensemble import RandomForestClassifier
from sklearn import model_selection
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from pandas.api.types import CategoricalDtype

## Enable Algorithm Options

In [38]:
# Enable Algorithms
enable_model_xgboost = True
enable_model_randomForest = True
enable_multiclass_model = True

predict_xgboost = True
predict_randomForest = True


#Debug
verbose_level=1
#Multiclass classification, binary if falase
multiclass = False
over_sample = True

# Datafile
#inputfile = 'CKME136X10_2018_Data_CTF.csv'
if multiclass:
    inputfile_train_O = 'CKME136X10_2018_Data_CTFB_M_O_Train.csv'
    inputfile_train_U = 'CKME136X10_2018_Data_CTFB_M_U_Train.csv'
    inputfile_test = 'CKME136X10_2018_Data_CTFB_M_Test.csv'
else:
    inputfile_train_O = 'CKME136X10_2018_Data_CTFB_B_O_Train.csv'
    inputfile_train_U = 'CKME136X10_2018_Data_CTFB_B_U_Train.csv'
    inputfile_test = 'CKME136X10_2018_Data_CTFB_B_Test.csv'

if over_sample:
    datafile_train = inputfile_train_O
else:
    datafile_train = inputfile_train_U

datafile_test = inputfile_test


#file_input = 'NCDB_FULL_Removed_All_Missing_Values_Binary_Class_Transformed.csv'

model_max_iter = 1000
datestr = 'dec_04_binary_run_1000_BO'

# Model File Names for storage
file_random_forest = 'random_forest_'  + datestr + '.model'
file_xgboost = 'xgboost_'  + datestr + '.model'

## Boosting

### XGBoost

In [39]:
#df = pd.read_csv(file_input, engine = 'python')

#load data
df_test = pd.read_csv(datafile_test, engine = 'python')
df_train = pd.read_csv(datafile_train, engine = 'python')
df = df_train.copy()

print(df_test.head(2))
print(df_train.head(2))

Exception ignored in: <function DMatrix.__del__ at 0x000001CCB85C20D0>
Traceback (most recent call last):
  File "C:\Users\pradeep\Anaconda3\lib\site-packages\xgboost\core.py", line 482, in __del__
    if self.handle is not None:
AttributeError: 'DMatrix' object has no attribute 'handle'
Exception ignored in: <function DMatrix.__del__ at 0x000001CCB85C20D0>
Traceback (most recent call last):
  File "C:\Users\pradeep\Anaconda3\lib\site-packages\xgboost\core.py", line 482, in __del__
    if self.handle is not None:
AttributeError: 'DMatrix' object has no attribute 'handle'


   C_YEAR  C_MNTH  C_WDAY  C_HOUR  C_VEHS  C_CONF  C_RCFG  C_WTHR  C_RSUR  \
0       1       5       5       2       3      21       2       1       1   
1      14       2       6       1       4      24       1       1       1   

   C_RALN  C_TRAF  V_TYPE  V_YEAR  P_SEX  P_AGE  P_PSN  P_SAFE  P_USER  P_ISEV  
0       1       1       6       2      1     34      1       2       1       0  
1       2       7       1       4      0     47      3       2       2       1  
   C_YEAR  C_MNTH  C_WDAY  C_HOUR  C_VEHS  C_CONF  C_RCFG  C_WTHR  C_RSUR  \
0       3       5       4       3       2      51       1       1       1   
1      16       3       1       1       2      51       1       1       1   

   C_RALN  C_TRAF  V_TYPE  V_YEAR  P_SEX  P_AGE  P_PSN  P_SAFE  P_USER  P_ISEV  
0       1       7       1       3      1     18      1       2       2       1  
1       1       7       6       4      1     19      1       2       1       1  


In [40]:
df.shape

(3534154, 19)

In [41]:
# split data into X and y
#X = df.iloc[:,0:16]
#Y = df.iloc[:,-1]

In [42]:
#type(X)

In [43]:
#type(Y)

In [7]:
print(df_test.isnull().sum().sum())
print(df_train.isnull().sum().sum())

0
0


In [8]:
print(df_test[df_test.columns].apply(lambda x: x.astype(str).str.contains('[^0-9]')).sum().sum())
print(df_train[df_train.columns].apply(lambda x: x.astype(str).str.contains('[^0-9]')).sum().sum())

0
0


In [19]:
df_test_cat = df_test.astype('int').copy()
df_train_cat = df_train.astype('int').copy()

df_test_cat = df_test_cat.astype('category')
df_train_cat = df_train_cat.astype('category')

# convert to the correct type
df_train_cat['C_YEAR'] = df_train_cat['C_YEAR'].astype(CategoricalDtype(ordered=True))
df_train_cat['C_MNTH'] = df_train_cat['C_MNTH'].astype(CategoricalDtype(ordered=True))
df_train_cat['C_WDAY'] = df_train_cat['C_WDAY'].astype(CategoricalDtype(ordered=True))
df_train_cat['C_HOUR'] = df_train_cat['C_HOUR'].astype(CategoricalDtype(ordered=True))
df_train_cat['C_VEHS'] = df_train_cat['C_VEHS'].astype('int')
df_train_cat['V_YEAR'] = df_train_cat['V_YEAR'].astype(CategoricalDtype(ordered=True))
df_train_cat['P_PSN'] = df_train_cat['P_PSN'].astype(CategoricalDtype(ordered=True))
df_train_cat['P_AGE'] = df_train_cat['P_AGE'].astype('int')
df_train_cat['P_ISEV'] = df_train_cat['P_ISEV'].astype('int')

print(df_train_cat.info())

# convert to the correct type
df_test_cat['C_YEAR'] = df_test_cat['C_YEAR'].astype(CategoricalDtype(ordered=True))
df_test_cat['C_MNTH'] = df_test_cat['C_MNTH'].astype(CategoricalDtype(ordered=True))
df_test_cat['C_WDAY'] = df_test_cat['C_WDAY'].astype(CategoricalDtype(ordered=True))
df_test_cat['C_HOUR'] = df_test_cat['C_HOUR'].astype(CategoricalDtype(ordered=True))
df_test_cat['C_VEHS'] = df_test_cat['C_VEHS'].astype('int')
df_test_cat['V_YEAR'] = df_test_cat['V_YEAR'].astype(CategoricalDtype(ordered=True))
df_test_cat['P_PSN'] = df_test_cat['P_PSN'].astype(CategoricalDtype(ordered=True))
df_test_cat['P_AGE'] = df_test_cat['P_AGE'].astype('int')
df_test_cat['P_ISEV'] = df_test_cat['P_ISEV'].astype('int')
print(df_test_cat.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3534154 entries, 0 to 3534153
Data columns (total 19 columns):
C_YEAR    category
C_MNTH    category
C_WDAY    category
C_HOUR    category
C_VEHS    int32
C_CONF    category
C_RCFG    category
C_WTHR    category
C_RSUR    category
C_RALN    category
C_TRAF    category
V_TYPE    category
V_YEAR    category
P_SEX     category
P_AGE     int32
P_PSN     category
P_SAFE    category
P_USER    category
P_ISEV    int32
dtypes: category(16), int32(3)
memory usage: 94.4 MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1300968 entries, 0 to 1300967
Data columns (total 19 columns):
C_YEAR    1300968 non-null category
C_MNTH    1300968 non-null category
C_WDAY    1300968 non-null category
C_HOUR    1300968 non-null category
C_VEHS    1300968 non-null int32
C_CONF    1300968 non-null category
C_RCFG    1300968 non-null category
C_WTHR    1300968 non-null category
C_RSUR    1300968 non-null category
C_RALN    1300968 non-null category
C_TRAF 

In [44]:
total_test_Rows = df_test_cat.index.size
print("Number of Rows in test data: {}".format(total_test_Rows))

total_train_Rows = df_train_cat.index.size
print("Number of Rows in train data: {}".format(total_train_Rows))

Number of Rows in test data: 1300968
Number of Rows in train data: 3534154


In [45]:
df_test_cat = df_test.astype('int').copy()
df_train_cat = df_train.astype('int').copy()

In [46]:
# split data into X and y
X_train = df_train_cat.iloc[:,0:18]
Y_train = df_train_cat.iloc[:,-1]

# split data into X and y
X_test = df_test_cat.iloc[:,0:18]
Y_test = df_test_cat.iloc[:,-1]

,C_YEAR,C_MNTH,C_WDAY,C_HOUR,C_VEHS,C_CONF,C_RCFG,C_WTHR,C_RSUR,C_RALN,C_TRAF,V_TYPE,V_YEAR,P_SEX,P_AGE,P_PSN,P_SAFE,P_USER
0,3,5,4,3,2,51,1,1,1,1,7,1,3,1,18,1,2,2
1,16,3,1,1,2,51,1,1,1,1,7,6,4,1,19,1,2,1
2,14,6,2,3,1,6,1,1,5,3,7,14,4,1,29,1,3,3
3,10,2,7,4,2,33,2,2,1,3,2,6,4,1,22,1,2,1
4,14,11,2,3,1,4,1,1,3,1,7,1,4,0,28,1,2,2
5,16,6,7,2,1,6,1,1,1,3,7,14,4,1,53,1,3,3
6,18,5,7,2,3,21,2,1,1,1,7,1,5,0,58,1,2,2
7,10,12,4,3,1,4,1,1,4,1,7,1,4,1,16,1,4,1
8,2,5,5,3,4,21,2,1,1,1,1,1,3,0,48,1,2,1
9,10,12,6,1,1,32,1,2,3,1,7,1,4,0,25,1,2,1


In [47]:
#Split between data and class for training
#Y_train = df_train_cat[df_train_cat.columns[-1]]
#X_train = df_train_cat[df_train_cat.columns[0:df_train_cat.columns.size -1]]

#Y_test = df_test_cat[df_test_cat.columns[-1]]
#X_test = df_test_cat[df_test_cat.columns[0:df_test_cat.columns.size -1]]

In [48]:
print(Y_train.unique())

[1 0]


In [49]:
print(X_train.head(3))

   C_YEAR  C_MNTH  C_WDAY  C_HOUR  C_VEHS  C_CONF  C_RCFG  C_WTHR  C_RSUR  \
0       3       5       4       3       2      51       1       1       1   
1      16       3       1       1       2      51       1       1       1   
2      14       6       2       3       1       6       1       1       5   

   C_RALN  C_TRAF  V_TYPE  V_YEAR  P_SEX  P_AGE  P_PSN  P_SAFE  P_USER  
0       1       7       1       3      1     18      1       2       2  
1       1       7       6       4      1     19      1       2       1  
2       3       7      14       4      1     29      1       3       3  


In [50]:
#seed = 10
#test_size = 0.33
#X_train, X_test, y_train, y_test = model_selection.train_test_split(X, Y, test_size=test_size, random_state=seed)

In [85]:
# fit model no training data
if (enable_model_xgboost):
    t_ =  time.time()
    print(time.asctime( time.localtime(t_) ))
    
    nX_train = np.array(X_train)
    nY_train = np.array(Y_train)
    
    model = xgboost.XGBClassifier(silent=False, n_jobs=10)
    #model.fit(X_train, Y_train)
    model.fit(nX_train, nY_train)
    
    # save model to file
    pickle.dump(model, open(file_xgboost, "wb"))
    
    t_ =  time.time()
    print(time.asctime( time.localtime(t_) ))

Mon Dec  3 10:12:07 2018
[10:12:09] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[10:12:10] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[10:12:11] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[10:12:11] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[10:12:12] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[10:12:13] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[10:12:13] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[10:12:14] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[10:12:15] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra no

[10:12:57] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[10:12:57] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[10:12:58] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[10:12:59] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[10:12:59] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[10:13:00] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[10:13:01] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[10:13:01] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[10:13:02] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_

In [86]:
X_test.isnull().sum()

C_YEAR    0
C_MNTH    0
C_WDAY    0
C_HOUR    0
C_VEHS    0
C_CONF    0
C_RCFG    0
C_WTHR    0
C_RSUR    0
C_RALN    0
C_TRAF    0
V_TYPE    0
V_YEAR    0
P_SEX     0
P_AGE     0
P_PSN     0
P_SAFE    0
P_USER    0
dtype: int64

### Predict XGBoost

In [87]:
nX_test = np.array(X_test)
nY_test = np.array(Y_test)
print(nX_test)

[[ 1  5  5 ...  1  2  1]
 [14  2  6 ...  3  2  2]
 [ 5  8  5 ...  2  2  2]
 ...
 [ 6  6  7 ...  1  2  2]
 [10  2  3 ...  1  2  1]
 [ 3 10  1 ...  1  7  1]]


In [88]:
#predictions for test data
if (predict_xgboost):
    # load model from file
    loaded_model = pickle.load(open(file_xgboost, "rb"))
    
    # make predictions for test data
    y_pred = loaded_model.predict(nX_test)
    print(y_pred)
    predictions = [round(value) for value in y_pred]
    # evaluate predictions
    accuracy = accuracy_score(nY_test, predictions)
    print("Accuracy: %.2f%%" % (accuracy * 100.0))

C:\Users\pradeep\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


[0 1 0 ... 1 1 0]
Accuracy: 66.93%


In [79]:
type(X_test)

pandas.core.frame.DataFrame

## Bagging Ensemble

### Random Forest

In [58]:
df_test_cat = df_test_cat.astype('category')
df_train_cat = df_train_cat.astype('category')

# convert to the correct type
df_train_cat['C_YEAR'] = df_train_cat['C_YEAR'].astype(CategoricalDtype(ordered=True))
df_train_cat['C_MNTH'] = df_train_cat['C_MNTH'].astype(CategoricalDtype(ordered=True))
df_train_cat['C_WDAY'] = df_train_cat['C_WDAY'].astype(CategoricalDtype(ordered=True))
df_train_cat['C_HOUR'] = df_train_cat['C_HOUR'].astype(CategoricalDtype(ordered=True))
df_train_cat['C_VEHS'] = df_train_cat['C_VEHS'].astype('int')
df_train_cat['V_YEAR'] = df_train_cat['V_YEAR'].astype(CategoricalDtype(ordered=True))
df_train_cat['P_PSN'] = df_train_cat['P_PSN'].astype(CategoricalDtype(ordered=True))
df_train_cat['P_AGE'] = df_train_cat['P_AGE'].astype('int')
df_train_cat['P_ISEV'] = df_train_cat['P_ISEV'].astype('int')

print(df_train_cat.info())

# convert to the correct type
df_test_cat['C_YEAR'] = df_test_cat['C_YEAR'].astype(CategoricalDtype(ordered=True))
df_test_cat['C_MNTH'] = df_test_cat['C_MNTH'].astype(CategoricalDtype(ordered=True))
df_test_cat['C_WDAY'] = df_test_cat['C_WDAY'].astype(CategoricalDtype(ordered=True))
df_test_cat['C_HOUR'] = df_test_cat['C_HOUR'].astype(CategoricalDtype(ordered=True))
df_test_cat['C_VEHS'] = df_test_cat['C_VEHS'].astype('int')
df_test_cat['V_YEAR'] = df_test_cat['V_YEAR'].astype(CategoricalDtype(ordered=True))
df_test_cat['P_PSN'] = df_test_cat['P_PSN'].astype(CategoricalDtype(ordered=True))
df_test_cat['P_AGE'] = df_test_cat['P_AGE'].astype('int')
df_test_cat['P_ISEV'] = df_test_cat['P_ISEV'].astype('int')
print(df_test_cat.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3534154 entries, 0 to 3534153
Data columns (total 19 columns):
C_YEAR    category
C_MNTH    category
C_WDAY    category
C_HOUR    category
C_VEHS    int32
C_CONF    category
C_RCFG    category
C_WTHR    category
C_RSUR    category
C_RALN    category
C_TRAF    category
V_TYPE    category
V_YEAR    category
P_SEX     category
P_AGE     int32
P_PSN     category
P_SAFE    category
P_USER    category
P_ISEV    int32
dtypes: category(16), int32(3)
memory usage: 94.4 MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1300968 entries, 0 to 1300967
Data columns (total 19 columns):
C_YEAR    1300968 non-null category
C_MNTH    1300968 non-null category
C_WDAY    1300968 non-null category
C_HOUR    1300968 non-null category
C_VEHS    1300968 non-null int32
C_CONF    1300968 non-null category
C_RCFG    1300968 non-null category
C_WTHR    1300968 non-null category
C_RSUR    1300968 non-null category
C_RALN    1300968 non-null category
C_TRAF 

In [59]:
if (enable_model_randomForest):
    print("Ensemble (Bagging): Random Forest: Start")
    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))
    forest = RandomForestClassifier(criterion='entropy', n_estimators=50, random_state=0, n_jobs=10, verbose=verbose_level)
    print("Ensemble (Bagging): Random Forest: Fit")
    forest.fit(X_train, Y_train)
    
    # save model to file
    pickle.dump(model, open(file_random_forest, "wb"))
    
    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))
    
    print("Ensemble (Bagging): Random Forest: End")

Ensemble (Bagging): Random Forest: Start
Mon Dec  3 09:35:26 2018
Ensemble (Bagging): Random Forest: Fit


[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:  1.8min


Mon Dec  3 09:38:23 2018
Ensemble (Bagging): Random Forest: End


[Parallel(n_jobs=10)]: Done  50 out of  50 | elapsed:  2.9min finished


In [61]:
#predictions for test data
if (predict_randomForest):
    
    # load model from file
    loaded_model = pickle.load(open(file_random_forest, "rb"))
    
    print("Ensemble (Bagging): Random Forest: Predict")
    y_pred = forest.predict(X_test)
    
    print('Accuracy of RandomForest classifier on train set: {:.2f}'.format(forest.score(X_train, Y_train)))
    print('Accuracy of RandomForest classifier on test set: {:.2f}'.format(forest.score(X_test, Y_test)))
    
    print("Ensemble (Bagging): Random Forest: Confusion Matrix")
    cnf_matrix_rf = confusion_matrix(Y_test, y_pred)
    print(cnf_matrix_rf)
    
    print("Ensemble (Bagging): Random Forest: Classification Report")
    print(classification_report(Y_test,y_pred))

Ensemble (Bagging): Random Forest: Predict


[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:    7.9s
[Parallel(n_jobs=10)]: Done  50 out of  50 | elapsed:   11.8s finished
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:   19.3s
[Parallel(n_jobs=10)]: Done  50 out of  50 | elapsed:   31.8s finished


Accuracy of RandomForest classifier on train set: 1.00


[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:    7.8s
[Parallel(n_jobs=10)]: Done  50 out of  50 | elapsed:   11.7s finished


Accuracy of RandomForest classifier on test set: 0.67
Ensemble (Bagging): Random Forest: Confusion Matrix
[[356810 186839]
 [248587 508732]]
Ensemble (Bagging): Random Forest: Classification Report
             precision    recall  f1-score   support

          0       0.59      0.66      0.62    543649
          1       0.73      0.67      0.70    757319

avg / total       0.67      0.67      0.67   1300968

